# Data Ingestion
This notebook collects data from different sources. Preprocesses as necessary and stores them as flat comma seperated files (csv). These files form the database of our system. A file consists of all the matches with players played and not played. A folder players is created where individual player data resides as seperate files. 


In [0]:
# Installing neccesary libraries
!pip install pyyaml # Used to read and parse yaml files.
!pip install python-espncricinfo # espncricinfor python api.

     |████████████████████████████████| 358kB 3.6MB/s 


In [1]:
# Getting required libraries
import os # handles os tasks
import pandas as pd # load, processes and stores data as 2d arrays. 
import numpy as np # Support for numerical operations
import yaml # reads yamls
from espncricinfo.match import Match # Gets information about Match
from espncricinfo.player import Player # Get infromation about Player
import re # regex to process strings

ModuleNotFoundError: ignored

# Creating Supporting Dataset that is used to reference values like name of player, playing role etc


In [0]:
all_data = pd.read_csv('https://query.data.world/s/pv2g46dl6iy5rf3icilntethshfsw7') # reading csv from url
columns = ['ID', 'NAME', 'COUNTRY', 'Age', 'Batting style',
           'Bowling style'] # Specifying column names
is_international = all_data['Major teams'].str.contains("'Pakistan,'") # Check if player plays Internationally
is_alive = all_data['Died'] == 'Alive' # Check if player is alive
is_Pakistani = all_data['COUNTRY'] == 'Pakistan' # Check if plays for Pakistan
mask = is_international & is_alive & is_Pakistani # Combining above 3 conditions
cleaned_data = all_data[mask][columns] # selected dataset based on mask and given columns
player_roles = pd.read_csv('https://query.data.world/s/zd3yb4g3zqzjhuhbkvpovsdninrbdv')[['ID', 'Playing role']].dropna() # Reading another url to get Playing roles
bio = pd.merge(cleaned_data, player_roles, on = ['ID']) # Merging both datasets using ID
bio.head() # Print first 5 items

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (101,107,108,140,141,145,146,147,153,154,158,159,160) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,16,20,21,22,24,25,26,27,28,29,30,31,32,45,46,59,60,73,74,87,88,90,91,92,95,96,97,98,99,100,101,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,NAME,COUNTRY,Age,Batting style,Bowling style,Playing role
0,43860,Zulqarnain Haider,Pakistan,33.0,Right-hand bat,NaN,Wicketkeeper batsman
1,43871,Zulfiqar Babar,Pakistan,41.0,Right-hand bat,Slow left-arm orthodox,Bowler
2,43692,Zafar Iqbal,Pakistan,50.0,Right-hand bat,Right-arm fast-medium,Allrounder
3,39037,Azhar Ali,Pakistan,34.0,Right-hand bat,Legbreak,Top-order batsman
4,278491,Awais Zia,Pakistan,33.0,Left-hand bat,Right-arm offbreak,Opening batsman


In [0]:
# This code selects columns related to batting or bowling and three international formats
new_cols = []
for col in all_data.columns: 
  if ('BOWLING' in col) or ('BATTING' in col):
    if ('Tests' in col) or ('ODIs' in col) or ('T20Is' in col): 
      new_cols.append(col)
new_cols.append('ID') # Add ID so to allow join
stats = all_data[new_cols] # Selecting the columns 
df = pd.merge(bio, stats, on = ['ID']).fillna(0) # merging datasets, and filling na values with 0
df.head() # printing first 5 entries

,ID,NAME,COUNTRY,Age,Batting style,Bowling style,Playing role,BATTING_Tests_Mat,BATTING_Tests_Inns,BATTING_Tests_NO,BATTING_Tests_Runs,BATTING_Tests_HS,BATTING_Tests_Ave,BATTING_Tests_BF,BATTING_Tests_SR,BATTING_Tests_100,BATTING_Tests_50,BATTING_Tests_4s,BATTING_Tests_6s,BATTING_Tests_Ct,BATTING_Tests_St,BATTING_ODIs_Mat,BATTING_ODIs_Inns,BATTING_ODIs_NO,BATTING_ODIs_Runs,BATTING_ODIs_HS,BATTING_ODIs_Ave,BATTING_ODIs_BF,BATTING_ODIs_SR,BATTING_ODIs_100,BATTING_ODIs_50,BATTING_ODIs_4s,BATTING_ODIs_6s,BATTING_ODIs_Ct,BATTING_ODIs_St,BATTING_T20Is_Mat,BATTING_T20Is_Inns,BATTING_T20Is_NO,BATTING_T20Is_Runs,BATTING_T20Is_HS,...,BATTING_T20Is_St,BOWLING_Tests_Mat,BOWLING_Tests_Inns,BOWLING_Tests_Balls,BOWLING_Tests_Runs,BOWLING_Tests_Wkts,BOWLING_Tests_BBI,BOWLING_Tests_BBM,BOWLING_Tests_Ave,BOWLING_Tests_Econ,BOWLING_Tests_SR,BOWLING_Tests_4w,BOWLING_Tests_5w,BOWLING_Tests_10,BOWLING_ODIs_Mat,BOWLING_ODIs_Inns,BOWLING_ODIs_Balls,BOWLING_ODIs_Runs,BOWLING_ODIs_Wkts,BOWLING_ODIs_BBI,BOWLING_ODIs_BBM,BOWLING_ODIs_Ave,BOWLING_ODIs_Econ,BOWLING_ODIs_SR,BOWLING_ODIs_4w,BOWLING_ODIs_5w,BOWLING_ODIs_10,BOWLING_T20Is_Mat,BOWLING_T20Is_Inns,BOWLING_T20Is_Balls,BOWLING_T20Is_Runs,BOWLING_T20Is_Wkts,BOWLING_T20Is_BBI,BOWLING_T20Is_BBM,BOWLING_T20Is_Ave,BOWLING_T20Is_Econ,BOWLING_T20Is_SR,BOWLING_T20Is_4w,BOWLING_T20Is_5w,BOWLING_T20Is_10
0,43860,Zulqarnain Haider,Pakistan,33.0,Right-hand bat,0,Wicketkeeper batsman,1.0,2.0,0.0,88.0,88,44.00,201.0,43.78,0.0,1.0,15.0,0.0,2.0,0.0,4.0,4.0,2.0,48.0,19*,24.0,68.0,70.58,0.0,0.0,3.0,0.0,1.0,1.0,3.0,3.0,0.0,23.0,17,...,1.0,1.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0
1,43871,Zulfiqar Babar,Pakistan,41.0,Right-hand bat,Slow left-arm orthodox,Bowler,15.0,18.0,9.0,144.0,56,16.00,207.0,69.56,0.0,1.0,15.0,4.0,4.0,0.0,5.0,5.0,3.0,35.0,14*,17.5,40.0,87.50,0.0,0.0,2.0,1.0,0.0,0.0,7.0,3.0,3.0,27.0,13*,...,0.0,15.0,28.0,4478.0,2129.0,54.0,May-74,8/233,39.42,2.85,82.9,2.0,2.0,0.0,5.0,5.0,294.0,246.0,4.0,Feb-52,Feb-52,61.50,5.02,73.5,0.0,0.0,0.0,7.0,7.0,156.0,185.0,12.0,Mar-23,Mar-23,15.41,7.11,13.0,0.0,0.0,0.0
2,43692,Zafar Iqbal,Pakistan,50.0,Right-hand bat,Right-arm fast-medium,Allrounder,0.0,0.0,0.0,0.0,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0,0.0,48.0,18,8.0,62.0,77.41,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,8.0,7.0,198.0,137.0,3.0,Feb-37,Feb-37,45.66,4.15,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0
3,39037,Azhar Ali,Pakistan,34.0,Right-hand bat,Legbreak,Top-order batsman,73.0,139.0,8.0,5669.0,302*,43.27,13686.0,41.42,15.0,31.0,524.0,16.0,61.0,0.0,53.0,53.0,3.0,1845.0,102,36.9,2478.0,74.45,3.0,12.0,168.0,14.0,8.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,73.0,34.0,848.0,602.0,8.0,Feb-35,Feb-49,75.25,4.25,106.0,0.0,0.0,0.0,53.0,14.0,258.0,260.0,4.0,Feb-26,Feb-26,65.00,6.04,64.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0
4,278491,Awais Zia,Pakistan,33.0,Left-hand bat,Right-arm offbreak,Opening batsman,0.0,0.0,0.0,0.0,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,70.0,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.0,0.0,0.0,0.0


In [0]:
# Stores the dataframe processed in last cell on their match types and player types
path = '/content/drive/My Drive/Freelancer/Player Prediction/Excel Files' # Path to store all data
os.chdir(path) # Changing current working directory
match_types = ['Tests', 'T20Is', 'ODIs'] # Match type we care about
player_types = ['BOWLING', 'BATTING'] # Player types required by us
static_columns = df.columns[:7].values # These columns remains same for all players
for player_type in player_types: # looping over player types
  for match_type in match_types: # looping over match types
    if not os.path.isfile(match_type + '_' + player_type + '.csv'): # check if file exists
      selected_cols = [] # dynamic columns depending upon match and player type
      for col in df.columns: # looping over columns of dataframe
        if match_type in col and player_type in col: # check if column is required
          selected_cols.append(col) # populate dynamic columns list
      new_cols = np.concatenate((static_columns, selected_cols)) # merge static and dynamic columns
      this_df = df[new_cols] # Select the selected columns
      selected_cols = [col.split("_")[-1] for col in selected_cols] # renaming column names for user ease
      this_df.columns = np.concatenate((static_columns, selected_cols)) # setting column names with renamed column names
      this_df = this_df[(this_df.Mat > 5) & (this_df.Inns > 5)] # Select players with at least 5 matches played
      this_df.to_csv(match_type + '_' + player_type + '.csv', index=False) # Saves the file to disk

The supporting dataset is ready and saved to disk

# This Dataset is used to get matches played by team Pakistan against all others. This will serve bases for whole project

In [0]:
os.chdir('/content') # changing current working directory

In [0]:
!wget -q https://cricsheet.org/downloads/odis.zip # Getting dataset files

In [0]:
!unzip -q odis.zip -d matches # extracting files

In [0]:
# Loads dataset, it contains all matches mentioned in the dataset
matches = pd.read_csv(r"matches/README.txt", sep=' - ', skiprows=24, 
                      names=['date', '1', '2', 'gender', 'id', 'match'],
                      usecols=['date', 'gender', 'id', 'match']) 
matches = matches[matches.match.str.contains('Pakistan', case = False)] # Selecting matches with team Pakistan
matches = matches[matches.gender == 'male'].drop(['gender'], axis =1) # Selecting only male matches
matches = matches.reset_index(drop = True) # cleaning
matches['Opposition'] = matches.match.map(lambda x: x.split(" vs ")[0] if x.split(" vs ")[0] != 'Pakistan' else x.split(" vs ")[1]) # Getting the oppoistion team
matches.drop('match', axis = 1, inplace = True) # dropping unnecessary column
matches.date = pd.to_datetime(matches.date) # Performance improvements
matches.sort_values(by=['date'], inplace=True, ascending=True) # Sorting by date
matches.date = matches.date.astype(str) # Performance improvements
print("Total number of matches:", matches.shape[0]) # printing the fetched number of matches 
matches.head() # Print first 5 matches: dates, ids and oppoistions

# Note: These ids are set by ESPNCricInfo. With id corresponding match can be retrieved.

Total number of matches: 272


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,date,id,Opposition
271,2006-02-06,235831,India
270,2006-02-11,236520,India
269,2006-02-13,236809,India
268,2006-02-16,237222,India
267,2006-02-19,237571,India


In [0]:

def get_player(player_dict):
    '''
    A function that returns 
    player name, player id and role
    given a dictionary object containing
    the information.
    This dict is retrieved using espn api.
    This dict contains all players played in match
    '''
    id = player_dict['object_id'] # Get player_id
    name = player_dict['known_as'] # Get player_name
    playing_role = Player(id).playing_role['name'] # Get playing_role
    return name, id, playing_role

# Converting the scalar function to vectorized function
# to enable parallel processing.
vget_player = np.vectorize(get_player,otypes = [str, int, str])

def extract(date, id, opposition):
      '''
      Given date, id of the match and opposition
      returns 
      Player names, ids, roles
      winning status: whether team won
      series and match ids
      opposition team
      date and venue match played at
      '''
      m = Match(id) # Retireves match inforamtion using espn api
      win = int(m.match_winner[:3] == 'PAK') # calculates whether team won
      venue = m.town_name # Venue where match played
      series_id = m.series_id # Series id
      if m.team_1_id == '7': # Check if team 1 is Pakistan
        players = m.team_1_players # Retrieve players for this team
      elif m.team_2_id == '7': # Check if team 2 is Pakistan
        players = m.team_2_players # Retrieve players for this team
      names, player_ids, playing_roles = vget_player(players) # Parallelzed function call
      
      # ERROR HANDLING: if elements of array are 11 append na to reach 12 length. 
      #--------------------------------------------------------------------------
      if names.shape[0] == 11: 
        names = np.append(names, [np.nan], axis = 0)
        player_ids = np.append(player_ids, [np.nan], axis = 0)
        playing_roles = np.append(playing_roles, [np.nan], axis = 0)
      #--------------------------------------------------------------------------
      
      #Creating arrays from scalar values
      #--------------------------------------------------------------------------
      match_ids = np.asarray([id]*names.shape[0]).astype(int)
      wins = np.asarray([win]*names.shape[0]).astype(float)
      series_ids = np.asarray([series_id]*names.shape[0])
      oppositions = np.asarray([opposition]*names.shape[0])
      dates = np.asarray([date]*names.shape[0])
      venues = np.asarray([venue]*names.shape[0])
      #--------------------------------------------------------------------------
      return names, player_ids, playing_roles, wins, series_ids, match_ids, oppositions, dates, venues

# Converting the scalar function to vectorized function
# to enable parallel processing.
# Signature specifies 3 scalar inputs and 9 array outputs 
vextract = np.vectorize(extract,
                        signature = '(),(),()->(n),(n),(n),(n),(n),(n),(n),(n),(n)')

def prepare(df):
  '''
  It handles the above two functions and call them to process data.

  Given dataframe with match id, date and opposition

  Return a new dataframe that includes 
  
  playing 11 with names, ids and roles of players
  Winnig status, opposition team
  date and venue played on
  match and series ids
  '''
  data = df.values # converting to numpy matrix
  dates, ids, oppositions = data[:,0], data[:,1], data[:,2] # retrieving 1-D arrays

  # Calling parallelized function to retrieve required arrays
  (names, player_ids, playing_roles,
   wins, series_ids,match_ids, 
   oppositions, dates, venues) = vextract(dates,
                                            ids, oppositions) 
  
  # returns the processed dataset
  return pd.DataFrame({'date': dates.flatten().astype(str),
                      'match': match_ids.flatten().astype(int),
                      'series': series_ids.flatten().astype(int),
                      'venue': venues.flatten().astype(str),
                      'opposition': oppositions.flatten().astype(str),
                      'player_id': player_ids.flatten().astype(int),
                      'player_name': names.flatten().astype(str),
                      'playing_role': playing_roles.flatten().astype(str),
                      'won': wins.flatten().astype(int)})

In [0]:
# players = prepare(matches) # Calling the function to get processed dataset 
# players.dropna(axis = 0, inplace = True) # Dropping nulls
# players.player_id = players.player_id.astype(int) # Performance improvement

In [0]:
players.head() # first 5 entries

,date,match,series,venue,opposition,player_id,player_name,playing_role,won,Played
0,2006-02-06,235831,14710,Peshawar,India,42683,Salman Butt,Batsman,PAK,1
1,2006-02-06,235831,14710,Peshawar,India,41028,Kamran Akmal,Wicketkeeper batsman,PAK,1
2,2006-02-06,235831,14710,Peshawar,India,42657,Shoaib Malik,Allrounder,PAK,1
3,2006-02-06,235831,14710,Peshawar,India,42639,Shahid Afridi,Allrounder,PAK,1
4,2006-02-06,235831,14710,Peshawar,India,43650,Mohammad Yousuf,Batsman,PAK,1


In [0]:
# path = '/content/drive/My Drive/Freelancer/Player Prediction/Excel Files' 
# os.chdir(path) # changing Current Working Directory
# players.to_csv('Players_in_matches.csv', index = False) # Storing intermediate dataset to disk

In [0]:
# Retrieve the dataset written recently.
path = '/content/drive/My Drive/Freelancer/Player Prediction/Excel Files'
os.chdir(path) # changing working directory
players = pd.read_csv('Players_in_matches.csv').dropna(axis = 0) # loading data and dropping na
players.dropna(inplace=True) # dropping na
players['played'] = [1] * players.shape[0] # Introducing new column Played. This variable will be used to generate probability for a player to play or not
players.won = players.won.map(lambda x: float(x == 'PAK')) # convert winning status to 1 or 0
players = players[players.playing_role != 'Unknown'] # dropping values with Playing_role Unknown
print(players.shape) # Printing dimensions of dataset
players.head() # Print first 5 entries

(2833, 10)


,date,match,series,venue,opposition,player_id,player_name,playing_role,won,played
0,2006-02-06,235831,14710,Peshawar,India,42683,Salman Butt,Batsman,1.0,1
1,2006-02-06,235831,14710,Peshawar,India,41028,Kamran Akmal,Wicketkeeper batsman,1.0,1
2,2006-02-06,235831,14710,Peshawar,India,42657,Shoaib Malik,Allrounder,1.0,1
3,2006-02-06,235831,14710,Peshawar,India,42639,Shahid Afridi,Allrounder,1.0,1
4,2006-02-06,235831,14710,Peshawar,India,43650,Mohammad Yousuf,Batsman,1.0,1


In [0]:
def get_not_played(players):
  '''
  A squad is announced for a series and players are selected from that squad
  for each match. Using this idea, the players who were in squad and didn't played
  are entered in the dataset with played value as 0

  Takes a dataset of players 
  Returns a dataset with players not played as well
  '''
  dataframe = players.copy() # creating copy of input dataframe
  cols = ['date', 'match', 'series', 'venue', 'opposition', 'won'] # static cols

  # Loading reference dataset to retrieve name and role of player if necessary
  #-----------------------------------------------------------------------------
  bowling = pd.read_csv('ODIs_BOWLING.csv', usecols=['ID', 'NAME','Playing role'])
  batting = pd.read_csv('ODIs_BATTING.csv', usecols=['ID', 'NAME','Playing role'])
  reference = pd.concat([bowling, batting]).drop_duplicates().set_index(['ID'])  
  #-----------------------------------------------------------------------------
  
  for _, group in players.groupby('series'): # Grouping dataframe by Series id 
    squad = group.player_id.unique() # Getting all players in a series
    for match_index, group2 in group.groupby('match'): # Grouping again by match_id
      team = group2.player_id.unique() # Getting players playing in current match
      not_played = [player for player in squad if player not in team] # Getting players did not played
      data = players[players.match == match_index].head(1) # Loading static columns
      for player in not_played: # looping over players
        data.player_id = player # setting player id
        try: 
          data.player_name = reference['NAME'][player] # setting player name
        except:
          data.player_name = Player(player).name # setting player name
        try: 
          data.playing_role = reference['Playing role'][player] # setting player role
        except:
          data.playing_role = Player(player).playing_role['name'] # setting player role
        data.played = 0 # Playing status
        dataframe = pd.concat([dataframe, data], ignore_index=True) # Appending to main dataset
  
  # Reducing possible playing roles to 4, Batsman, Bowler, Wicketkeeper, Allrounder
  # -----------------------------------------------------------------------------
  dataframe.playing_role = dataframe.playing_role.map(lambda x: 'Wicketkeeper' if ('Wicketkeeper' in x) else x)
  dataframe.playing_role = dataframe.playing_role.map(lambda x: 'Batsman' if ('batsman' in x) else x)
  dataframe.playing_role = dataframe.playing_role.map(lambda x: 'Allrounder' if ('allrounder' in x) else x)
  dataframe.playing_role = dataframe.playing_role.map(lambda x: 'Bowler' if ('bowler' in x) else x)
  # -----------------------------------------------------------------------------
  
  return dataframe

In [0]:
# players = get_not_played(players) # Calling function to get players who did not played
# players.head() # printing first 5

,date,match,series,venue,opposition,player_id,player_name,playing_role,won,played
0,2006-02-06,235831,14710,Peshawar,India,42683,Salman Butt,Batsman,1.0,1
1,2006-02-06,235831,14710,Peshawar,India,41028,Kamran Akmal,Wicketkeeper,1.0,1
2,2006-02-06,235831,14710,Peshawar,India,42657,Shoaib Malik,Allrounder,1.0,1
3,2006-02-06,235831,14710,Peshawar,India,42639,Shahid Afridi,Allrounder,1.0,1
4,2006-02-06,235831,14710,Peshawar,India,43650,Mohammad Yousuf,Batsman,1.0,1


In [0]:
# This code completes the name of teams as some of them are missed due to some bug
# teams = pd.read_csv(r'players/42639.csv', index_col=['Start Date']).Opposition.unique()
# def get_team(x):
#   l = [team for team in teams if x in team]
#   if len(l) != 0:
#     return l[0]
#   else:
#     return x
# players.opposition = players.oppoistion.map(lambda x: get_team)
# players = players.replace('Unite', 'U.A.E')

In [0]:
# path = '/content/drive/My Drive/Freelancer/Player Prediction/Excel Files'
# os.chdir(path) # Changing current working directory
# players.to_csv('Players_with_not_played_in_matches.csv', index = False) # Saving to disk

Till now base or main dataframe is processed and saved to disk


# Some Stats

In [0]:
path = '/content/drive/My Drive/Freelancer/Player Prediction/Excel Files'
os.chdir(path) # Changing current working directory
players = pd.read_csv('Players_with_not_played_in_matches.csv') # Loading from disk

In [0]:
players.playing_role.value_counts() # player counts by their role

Batsman         1434
Bowler          1274
Allrounder       754
Wicketkeeper     261
Name: playing_role, dtype: int64

In [0]:
players.won.value_counts() # entries with won status

0.0    1867
1.0    1856
Name: won, dtype: int64

In [0]:
players.played.value_counts() # Player count by played status

1    2833
0     890
Name: played, dtype: int64

In [0]:
players.player_id.nunique() # Number of players 

84

# Loading, processing and saving individual datasets for each player

In [0]:
def get_batting(id):
  '''
  This function retrieves player stats for batting.
  It uses espncricinfo site directly.
  '''
  # Loading from url
  df = pd.read_html('https://stats.espncricinfo.com/ci/engine/player/{}.html?class=2;template=results;type=batting;view=innings'.format(id))[3]
  
  # columns to use
  cols = ['Start Date', 'Opposition', 'Ground',
          'Runs', 'Mins', 'BF', 
          '4s', '6s', 'SR']
  # new names of columns
  new_cols = ['Start Date', 'Opposition', 'Ground',
          'Runs_scored', 'Mins', 'BF', 
          '4s', '6s', 'SR']
  df = df[cols] # selecting the selected columns
  df.columns = new_cols # renaming columns 
  cols = ['Runs_scored', 'Mins', 'BF', '4s', '6s', 'SR'] # columns to process
  
  # cleaning
  #-----------------------------------------------------------------------------
  df.replace('-', 0, inplace = True) # 
  df.Runs_scored = df.Runs_scored.astype(str).map(lambda x: 0 if x.isalpha() else x)
  df.Runs_scored = df.Runs_scored.astype(str).map(lambda x: re.sub("[^0-9]", "", x))
  df[cols] = df[cols].astype(float)
  df.Opposition = df.Opposition.map(lambda x: x[2:])
  #-----------------------------------------------------------------------------
  return df

def get_fielding(id):
  '''
  This function retrieves player stats for fielding.
  It uses espncricinfo site directly.
  '''
  # Loading from url
  df = pd.read_html('https://stats.espncricinfo.com/ci/engine/player/{}.html?class=2;template=results;type=fielding;view=innings'.format(id))[3]
  
  # columns to use
  cols = ['Start Date', 'Opposition', 'Ground',
          'Dis', 'Ct', 'St', 
          'Ct Wk', 'Ct Fi']
  df = df[cols] # selecting the selected columns
  cols = ['Dis', 'Ct', 'St', 'Ct Wk', 'Ct Fi',]# columns to process
  
  # cleaning
  #--------------------------------------------------------------------------
  df.replace('-', 0, inplace = True)
  df.Dis = df.Dis.astype(str).map(lambda x: 0 if x.isalpha() else x)
  df[cols] = df[cols].astype(int)
  df.Opposition = df.Opposition.map(lambda x: x[2:])
  #-----------------------------------------------------------------------------
  return df

def get_bowling(id):
  '''
  This function retrieves player stats for bowling.
  It uses espncricinfo site directly.
  '''
  # Loading from url
  df = pd.read_html('https://stats.espncricinfo.com/ci/engine/player/{}.html?class=2;template=results;type=bowling;view=innings'.format(id))[3]
  
  # columns to use
  cols = ['Start Date', 'Opposition', 'Ground',
          'Overs', 'Mdns', 'Runs', 
          'Wkts', 'Econ']
  # new names of columns
  new_cols = ['Start Date', 'Opposition', 'Ground',
          'Overs', 'Mdns', 'Runs_concieved', 
          'Wkts', 'Econ']
  
  df = df[cols] # selecting the selected columns
  df.columns = new_cols # renaming columns 
  cols = ['Overs', 'Mdns', 'Runs_concieved', 'Wkts', 'Econ'] # columns to process
  
  # cleaning
  #--------------------------------------------------------------------------
  df.replace('-', 0, inplace = True)
  df.Overs = df.Overs.astype(str).map(lambda x: 0 if x.isalpha() else x)
  df[cols] = df[cols].astype(float)
  df.Opposition = df.Opposition.map(lambda x: x[2:])
  #-----------------------------------------------------------------------------
  return df

def get_stats(id):
  '''
  Given id of the player.
  It returns complete performance history
  for each match of the player.

  It calls all the supporting functions to create dataframe
  '''
  df_fielding = get_fielding(id) # get fielding stats
  df_batting = get_batting(id) # get batting stats
  df_bowling = get_bowling(id) # get bowling stats
  
  # Merging frames to get single dataframe
  #-----------------------------------------------------------------------------
  df = pd.merge(df_fielding, df_batting, 
                on = ['Start Date', 'Opposition', 'Ground'])
  df = pd.merge(df, df_bowling, 
                on = ['Start Date', 'Opposition', 'Ground'])
  #-----------------------------------------------------------------------------
  
  df.to_csv('players/{}.csv'.format(id), index = False) # Saving to disk

# Converting the scalar function to vectorized function
# to enable parallel processing.
vget_stats = np.vectorize(get_stats) 

In [0]:
vget_stats(players.player_id.unique()) # Get stats for all players and store them in a folder.